
# <div style="text-align:center">2022년 봄학기 프로그래밍3 - 13주차 <br/><pandas 디버깅과 테스트></div>

<div style="text-align: right"> <b>강남대학교 인공지능융합공학부 데이터사이언스전공 곽찬희</b></div>
<div style="text-align: right"> <b>chk at kangnam dot ac dot kr</b></div>

---
### 공지사항
- 향후 수업 진행 일정
    - 10주차: 데이터 재구성 및 pandas 객체 병합
    - 11주차: 시계열 분석
    - 오늘!12주차: 인덱스 정렬 / 과제 진행
    - **13주차: 디버깅 및 테스트 / 과제 진행 및 중간 보고 제출(github 링크)**
    - 14주차: 발표 - github 를 이용한 10분 발표 / (동영상 제출 마감)
    - 15주차: 기말고사 및 학기 종료 (기말고사 형태는 추후 공지)
- 발표순서는 조 번호순 (동영상 시청 및 QnA)

    
<br/>

    

먼저 데이터를 load 해봅시다.

zip 파일 안에 csv 들을 불러오기 위해 zipfile 모듈을 사용합니다.

In [3]:
import pandas as pd
import numpy as np
import zipfile
url = 'archive.zip'

In [4]:
with zipfile.ZipFile(url) as z:
    print(z.namelist())
    kag = pd.read_csv(z.open('multipleChoiceResponses.csv'))
    df = kag.iloc[1:]

['SurveySchema.csv', 'freeFormResponses.csv', 'multipleChoiceResponses.csv']


C:\Users\K\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0,2,8,10,21,23,24,25,26,27,28,44,56,64,83,85,87,107,109,123,125,150,157,172,174,194,210,218,219,223,246,249,262,264,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,304,306,325,326,329,341,368,371,384,385,389,390,391,393,394) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
df.T

,1,2,3,4,5,6,7,8,9,10,...,23850,23851,23852,23853,23854,23855,23856,23857,23858,23859
Time from Start to Finish (seconds),710,434,718,621,731,1142,959,1758,641,751,...,820,683,57,122,348,575,131,370,36,502
Q1,Female,Male,Female,Male,Male,Male,Male,Male,Male,Male,...,Female,Male,Female,Female,Male,Male,Female,Male,Male,Male
Q1_OTHER_TEXT,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
Q2,45-49,30-34,30-34,35-39,22-24,25-29,35-39,18-21,25-29,30-34,...,18-21,22-24,18-21,30-34,30-34,45-49,25-29,22-24,25-29,25-29
Q3,United States of America,Indonesia,United States of America,United States of America,India,Colombia,Chile,India,Turkey,Hungary,...,India,Turkey,Turkey,Turkey,Turkey,France,Turkey,Turkey,United Kingdom of Great Britain and Northern I...,Spain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q50_Part_5,NaN,NaN,NaN,Not enough incentives to share my work,Not enough incentives to share my work,NaN,NaN,Not enough incentives to share my work,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q50_Part_6,NaN,NaN,NaN,NaN,NaN,I had never considered making my work easier f...,I had never considered making my work easier f...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q50_Part_7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q50_Part_8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.dtypes

Time from Start to Finish (seconds)    object
Q1                                     object
Q1_OTHER_TEXT                          object
Q2                                     object
Q3                                     object
                                        ...  
Q50_Part_5                             object
Q50_Part_6                             object
Q50_Part_7                             object
Q50_Part_8                             object
Q50_OTHER_TEXT                         object
Length: 395, dtype: object

SurveySchema 파일에는 각 문항에 대한 설명들이 들어가 있습니다. 

보통은 주어진 보기에서 고르는 경우가 많죠. 아래를 봅시다.

In [7]:
df.Q1.value_counts(dropna=False)

Male                       19430
Female                      4010
Prefer not to say            340
Prefer to self-describe       79
Name: Q1, dtype: int64

이제 컬럼명을 바꾸면서 데이터 정리를 해보겠습니다.

In [12]:
def tweak_kag(df):
    na_mask = df.Q9.isna()
    hide_mask = df.Q9.str.startswith('I do not').fillna(False)
    df = df[~na_mask & ~hide_mask]

    # q1 은 Gender 에 대한 응답
    q1 = (df.Q1
      .replace({'Prefer not to say': 'Another',
               'Prefer to self-describe': 'Another'})
      .rename('Gender')
    )

    # q2는 나이
    
    q2 = df.Q2.str.slice(0,2).astype(int).rename('Age')
    
    #q3에서는 미국, 인도, 중국을 제외한 국가를 Another 로 표기
    def limit_countries(val):
        if val in  {'United States of America', 'India', 'China'}:
            return val
        return 'Another'
    q3 = df.Q3.apply(limit_countries).rename('Country')

    #q4는 학력 코드화
    q4 = (df.Q4
     .replace({'Master’s degree': 18,
     'Bachelor’s degree': 16,
     'Doctoral degree': 20,
     'Some college/university study without earning a bachelor’s degree': 13,
     'Professional degree': 19,
     'I prefer not to answer': None,
     'No formal education past high school': 12})
     .fillna(11)
     .rename('Edu')
    )


    #q5에서는 전공 처리
    def only_cs_stat_val(val):
        if val not in {'cs', 'eng', 'stat'}:
            return 'another'
        return val

    q5 = (df.Q5
            .replace({
                'Computer science (software engineering, etc.)': 'cs',
                'Engineering (non-computer focused)': 'eng',
                'Mathematics or statistics': 'stat'})
             .apply(only_cs_stat_val)
             .rename('Studies'))
    
    # q6은 직업
    def limit_occupation(val):
        if val in {'Student', 'Data Scientist', 'Software Engineer', 'Not employed',
                  'Data Engineer'}:
            return val
        return 'Another'


    q6 = df.Q6.apply(limit_occupation).rename('Occupation')

    # q8은 경력
    q8 = (df.Q8
      .str.replace('+', '', regex=True)
      .str.split('-', expand=True)
      .iloc[:,0]
      .fillna(-1)
      .astype(int)
      .rename('Experience')
    )

    #q9는 임금수준
    q9 = (df.Q9
     .str.replace('+','')
     .str.replace(',','')
     .str.replace('500000', '500')
     .str.replace('I do not wish to disclose my approximate yearly compensation','')
     .str.split('-', expand=True)
     .iloc[:,0]
     .astype(int)
     .mul(1000)
     .rename('Salary'))
    return pd.concat([q1, q2, q3, q4, q5, q6, q8, q9], axis=1)

In [13]:
tweak_kag(df)

C:\Users\K\AppData\Local\Temp/ipykernel_14240/4157588000.py:73: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  q9 = (df.Q9


,Gender,Age,Country,Edu,Studies,Occupation,Experience,Salary
2,Male,30,Another,16.0,eng,Another,5,10000
3,Female,30,United States of America,18.0,cs,Data Scientist,0,0
5,Male,22,India,18.0,stat,Another,0,0
7,Male,35,Another,20.0,another,Another,10,10000
8,Male,18,India,18.0,another,Another,0,0
...,...,...,...,...,...,...,...,...
23844,Male,30,Another,18.0,cs,Software Engineer,10,90000
23845,Male,22,Another,18.0,stat,Student,0,0
23854,Male,30,Another,20.0,cs,Another,5,10000
23855,Male,45,Another,20.0,cs,Another,5,250000


In [14]:
tweak_kag(df).dtypes

C:\Users\K\AppData\Local\Temp/ipykernel_14240/4157588000.py:73: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  q9 = (df.Q9


Gender         object
Age             int32
Country        object
Edu           float64
Studies        object
Occupation     object
Experience      int32
Salary          int32
dtype: object

위와 같이 정리하면 조금 더 쉽게 분석이 가능합니다.

예를 들어, Salary 와 Experience 의 상관관계를 봅시다.

In [15]:
kag = tweak_kag(df)
(kag
    .groupby('Country')
    .apply(lambda g: g.Salary.corr(g.Experience))
)

C:\Users\K\AppData\Local\Temp/ipykernel_14240/4157588000.py:73: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  q9 = (df.Q9


Country
Another                     0.289827
China                       0.252974
India                       0.167335
United States of America    0.354125
dtype: float64

## Apply 성능
apply 는 편리한 함수이지만, 시간이 오래 걸리는(==비용이 많이 드는) 작업입니다.

이 비용을 어떻게 줄일 수 있을지 알아봅시다.

In [16]:
def limit_countries(val):
     if val in  {'United States of America', 'India', 'China'}:
         return val
     return 'Another'

In [26]:
%%timeit
q3 = df.Q3.apply(limit_countries).rename('Country')

In [19]:
%%timeit
other_values = df.Q3.value_counts().iloc[3:].index
q3_2 = df.Q3.replace(other_values, 'Another')
#replace 는 더 느립니다.

31 ms ± 185 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [22]:
%%timeit
values = {'United States of America', 'India', 'China'}
q3_3 = df.Q3.where(df.Q3.isin(values), 'Another')
#책에서는 이 부분이 훨씬 빨라진다고 했지만, 큰 차이는 없습니다.

1.55 ms ± 28.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [23]:
%%timeit
values = {'United States of America', 'India', 'China'}
q3_4 = pd.Series(np.where(df.Q3.isin(values), df.Q3, 'Another'), 
     index=df.index)
#마지막으로 np.where 를 통해 numpy 함수를 직접 써봅니다.

1.33 ms ± 23.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [27]:
q3.equals(q3_2)

NameError: name 'q3_2' is not defined

In [ ]:
q3.equals(q3_3)

In [ ]:
q3.equals(q3_4)

### Apply debug 하기

apply 에 적용할 함수에는 인자를 넘겨줄 수 있는데, 이 인자가 어떤 것인지 눈으로 확인하고 싶을 때가 있습니다.
그럴 때엔 아래와 같이 중간 단계의 함수를 만들면 편리합니다.

In [32]:
def limit_countries(val):
     if val in  {'United States of America', 'India', 'China'}:
         return val
     return 'Another'

In [33]:
q3 = df.Q3.apply(limit_countries).rename('Country')

In [34]:
def debug(something):
    # what is something? A cell, series, dataframe?
    print(type(something), something)
    1/0

In [35]:
q3.apply(debug)

<class 'str'> United States of America


ZeroDivisionError: division by zero

In [39]:
#글로벌 변수를 이용해, 해당 인자를 저장할 수도 있습니다.
the_item = None
def debug(something):
    global the_item
    the_item = something
    return something

In [37]:
_ = q3.apply(debug)

In [38]:
the_item

'Another'

## Improving Apply Performance with Dask, Pandarell, Swifter, and More

### How to do it...

In [47]:
!pip install pandarallel
!pip install swifter

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for swifter: filename=swifter-1.1.3-py3-none-any.whl size=13077 sha256=24bd972a88ed230bc5c8d966e4d4283f6ee73b56171395da21b44ba403b5eb28
  Stored in directory: c:\users\k\appdata\local\pip\cache\wheels\eb\11\ab\0b6c21c1ae17d57b6005df3f022fb7a6ce3c769cdaeb35f258
Successfully built swifter


In [42]:
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [43]:
def limit_countries(val):
     if val in  {'United States of America', 'India', 'China'}:
         return val
     return 'Another'

In [46]:
%%timeit
res_p = df.Q3.parallel_apply(limit_countries).rename('Country')

2.11 s ± 122 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
import swifter

In [49]:
%%timeit
res_s = df.Q3.swifter.apply(limit_countries).rename('Country')

Pandas Apply:   0%|          | 0/23859 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/23859 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/23859 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/23859 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/23859 [00:00<?, ?it/s]

...

Pandas Apply:   0%|          | 0/23859 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/23859 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/23859 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/23859 [00:00<?, ?it/s]

61.9 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [50]:
import dask

In [52]:
%%timeit
res_d = (dask.dataframe.from_pandas(
       df, npartitions=4)
   .map_partitions(lambda df: df.Q3.apply(limit_countries))
   .rename('Countries')
)

495 ms ± 24.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [53]:
np_fn = np.vectorize(limit_countries)

In [54]:
%%timeit
res_v = df.Q3.apply(np_fn).rename('Country')

287 ms ± 2.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [55]:
from numba import jit

In [56]:
@jit
def limit_countries2(val):
     if val in  ['United States of America', 'India', 'China']:
         return val
     return 'Another'

In [57]:
%%timeit
res_n = df.Q3.apply(limit_countries2).rename('Country')

68.2 ms ± 982 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## 코드 분석하기
우리는 jupyter를 사용하기에, 여러 방법들을 이용한 코드 분석이 가능합니다.


In [58]:
import zipfile
url = 'archive.zip'

In [59]:
with zipfile.ZipFile(url) as z:
    kag = pd.read_csv(z.open('multipleChoiceResponses.csv'))
    df = kag.iloc[1:]

C:\Users\K\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0,2,8,10,21,23,24,25,26,27,28,44,56,64,83,85,87,107,109,123,125,150,157,172,174,194,210,218,219,223,246,249,262,264,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,304,306,325,326,329,341,368,371,384,385,389,390,391,393,394) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [63]:
df.Q3.apply?
#help(df.Q3.apply)

Signature:
df.Q3.apply(
    func: 'AggFuncType',
    convert_dtype: 'bool' = True,
    args: 'tuple[Any, ...]' = (),
    **kwargs,
) -> 'FrameOrSeriesUnion'
Docstring:
Invoke function on values of Series.

Can be ufunc (a NumPy function that applies to the entire Series)
or a Python function that only works on single values.

Parameters
----------
func : function
    Python function or NumPy ufunc to apply.
convert_dtype : bool, default True
    Try to find better dtype for elementwise function results. If
    False, leave as dtype=object. Note that the dtype is always
    preserved for some extension array dtypes, such as Categorical.
args : tuple
    Positional arguments passed to func after the series value.
**kwargs
    Additional keyword arguments passed to func.

Returns
-------
Series or DataFrame
    If func returns a Series object the result will be a DataFrame.

See Also
--------
Series.map: For element-wise operations.
Series.agg: Only perform aggregating type operations.
Se

In [64]:
df.Q3.apply??

Signature:
df.Q3.apply(
    func: 'AggFuncType',
    convert_dtype: 'bool' = True,
    args: 'tuple[Any, ...]' = (),
    **kwargs,
) -> 'FrameOrSeriesUnion'
Source:   
    def apply(
        self,
        func: AggFuncType,
        convert_dtype: bool = True,
        args: tuple[Any, ...] = (),
        **kwargs,
    ) -> FrameOrSeriesUnion:
        """
        Invoke function on values of Series.

        Can be ufunc (a NumPy function that applies to the entire Series)
        or a Python function that only works on single values.

        Parameters
        ----------
        func : function
            Python function or NumPy ufunc to apply.
        convert_dtype : bool, default True
            Try to find better dtype for elementwise function results. If
            False, leave as dtype=object. Note that the dtype is always
            preserved for some extension array dtypes, such as Categorical.
        args : tuple
            Positional arguments passed to func after the se

In [65]:
import pandas.core.series
pandas.core.series.lib

<module 'pandas._libs.lib' from 'C:\\Users\\K\\anaconda3\\lib\\site-packages\\pandas\\_libs\\lib.cp38-win_amd64.pyd'>

In [66]:
pandas.core.series.lib.map_infer??

Docstring:
Substitute for np.vectorize with pandas-friendly dtype inference.

Parameters
----------
arr : ndarray
f : function
convert : bint
ignore_na : bint
    If True, NA values will not have f applied

Returns
-------
np.ndarray
Type:      builtin_function_or_method


In [69]:
import zipfile
url = 'archive.zip'

In [70]:
with zipfile.ZipFile(url) as z:
    kag = pd.read_csv(z.open('multipleChoiceResponses.csv'))
    df = kag.iloc[1:]

C:\Users\K\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0,2,8,10,21,23,24,25,26,27,28,44,56,64,83,85,87,107,109,123,125,150,157,172,174,194,210,218,219,223,246,249,262,264,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,304,306,325,326,329,341,368,371,384,385,389,390,391,393,394) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [78]:
def add1(x):
    return x + 1

In [79]:
df.Q3.apply(add1)

TypeError: can only concatenate str (not "int") to str

In [80]:
%debug

> c:\users\k\appdata\local\temp\ipykernel_14240\3352928176.py(2)add1()



ipdb>  p x


'United States of America'


ipdb>  q


In [73]:
from IPython.core.debugger import set_trace

In [74]:
def add1(x):
    set_trace()
    return x + 1

In [76]:
df.Q3.apply(add1)

> c:\users\k\appdata\local\temp\ipykernel_14240\1527908075.py(3)add1()



ipdb>  q


BdbQuit: 

## Great Expectations 활용해 데이터 무결성 보장하기
GE는 데이터의 속성을 캡처하고 정의하여 추후 데이터 무결성을 보장할 수 있는 도구입니다.

In [81]:
!pip install great-expectations

  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.6
    Uninstalling pyparsing-3.0.6:
      Successfully uninstalled pyparsing-3.0.6
  Attempting uninstall: notebook
    Found existing installation: notebook 6.4.6
    Uninstalling notebook-6.4.6:
      Successfully uninstalled notebook-6.4.6


ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\K\\AppData\\Local\\Temp\\pip-uninstall-4qf3vlqe\\jupyter-bundlerextension.exe'
Consider using the `--user` option or check the permissions.



In [84]:
kag = tweak_kag(df)

In [85]:
import great_expectations as ge
kag_ge = ge.from_pandas(kag)

In [86]:
sorted([x for x in set(dir(kag_ge)) - set(dir(kag))
    if not x.startswith('_')])

['add_citation',
 'append_expectation',
 'attempt_allowing_relative_error',
 'autoinspect',
 'batch_id',
 'batch_kwargs',
 'batch_markers',
 'batch_parameters',
 'caching',
 'column_aggregate_expectation',
 'column_map_expectation',
 'column_pair_map_expectation',
 'default_expectation_args',
 'discard_failing_expectations',
 'discard_subset_failing_expectations',
 'edit_expectation_suite',
 'expect_column_bootstrapped_ks_test_p_value_to_be_greater_than',
 'expect_column_chisquare_test_p_value_to_be_greater_than',
 'expect_column_distinct_values_to_be_in_set',
 'expect_column_distinct_values_to_contain_set',
 'expect_column_distinct_values_to_equal_set',
 'expect_column_kl_divergence_to_be_less_than',
 'expect_column_max_to_be_between',
 'expect_column_mean_to_be_between',
 'expect_column_median_to_be_between',
 'expect_column_min_to_be_between',
 'expect_column_most_common_value_to_be_in_set',
 'expect_column_pair_cramers_phi_value_to_be_less_than',
 'expect_column_pair_values_A_to_be

In [87]:
kag_ge.expect_column_to_exist('Salary')

{
  "success": true,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {},
  "meta": {}
}

In [88]:
kag_ge.expect_column_mean_to_be_between(
   'Salary', min_value=10_000, max_value=100_000)

{
  "success": true,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "observed_value": 43869.66102793441,
    "element_count": 15429,
    "missing_count": null,
    "missing_percent": null
  },
  "meta": {}
}

In [89]:
kag_ge.expect_column_values_to_be_between(
   'Salary', min_value=0, max_value=500_000)

{
  "success": true,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 15429,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {}
}

In [90]:
kag_ge.expect_column_values_to_not_be_null('Salary')

{
  "success": true,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 15429,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {}
}

In [91]:
kag_ge.expect_column_values_to_match_regex(
    'Country', r'America|India|Another|China')

{
  "success": true,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 15429,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {}
}

In [92]:
kag_ge.expect_column_values_to_be_of_type(
   'Salary', type_='int')

{
  "success": true,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "observed_value": "int32"
  },
  "meta": {}
}

In [93]:
kag_ge.save_expectation_suite('kaggle_expectations.json')

In [94]:
kag_ge.to_csv('kag.csv')
import json
ge.validate(ge.read_csv('kag.csv'), 
    expectation_suite=json.load(
        open('kaggle_expectations.json')))

{
  "results": [
    {
      "success": true,
      "exception_info": {
        "raised_exception": false,
        "exception_message": null,
        "exception_traceback": null
      },
      "result": {},
      "meta": {},
      "expectation_config": {
        "kwargs": {
          "column": "Salary"
        },
        "meta": {},
        "expectation_type": "expect_column_to_exist"
      }
    },
    {
      "success": true,
      "exception_info": {
        "raised_exception": false,
        "exception_message": null,
        "exception_traceback": null
      },
      "result": {
        "observed_value": 43869.66102793441,
        "element_count": 15429,
        "missing_count": null,
        "missing_percent": null
      },
      "meta": {},
      "expectation_config": {
        "kwargs": {
          "column": "Salary",
          "max_value": 100000,
          "min_value": 10000
        },
        "meta": {},
        "expectation_type": "expect_column_mean_to_be_between"
      }


### How it works...

## Using pytest with pandas

### How to do it...

### How it works...

### There's more...

## Generating Tests with Hypothesis

### How to do it...

### How it works...